In [2]:
%load_ext autoreload
%autoreload 2

from Shared.shared import *
from Shared.specific_CNB_sim import *

sim_name = f"no_gravity_decay"
sim_folder = f"sim_output/{sim_name}"
fig_folder = f"figures_local/{sim_name}"
Cl_folder = f"Shared/Cls"
Delta_folder = f"Shared/Deltas"
nu_m_range = jnp.load(f"{sim_folder}/neutrino_massrange_eV.npy")
nu_m_picks = jnp.array([0.01, 0.05, 0.1, 0.2, 0.3])*Params.eV
simdata = SimData(sim_folder)

# note: We must have realistic neutrino masses
m_light, m_mid, m_heavy = Physics.neutrino_masses(
    m_lightest=0.05*Params.eV, ordering="NO", args=Params())
print(m_light, m_mid, m_heavy)

def find_nearest(array, value):
    idx = jnp.argmin(jnp.abs(array - value))
    return idx, array[idx]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.05 0.05073657457889723 0.07080960386840193


In [4]:
Gamma_str ='2T'

angle_momentum_decay = jnp.load(
    f'{sim_folder}/allowed_decay_angles_and_momenta.npy')
decayed_neutrinos_z = jnp.load(
    f'{sim_folder}/decayed_neutrinos_z_{Gamma_str}.npy')
decayed_neutrinos_index_z = jnp.load(
    f'{sim_folder}/decayed_neutrinos_index_z_{Gamma_str}.npy', allow_pickle=True)

z_array = jnp.load(
    f'{sim_folder}/z_int_steps.npy')
neutrino_momenta = jnp.load(
    f'{sim_folder}/neutrino_momenta.npy')

ic(angle_momentum_decay.shape)
ic(decayed_neutrinos_z.shape)
ic(decayed_neutrinos_index_z.shape)
ic(z_array.shape)
ic(neutrino_momenta.shape)
print("END")

ic| angle_momentum_decay.shape: (1000, 1800, 2)
ic| decayed_neutrinos_z.shape: (99, 768000)
ic| decayed_neutrinos_index_z.shape: (99,)
ic| z_array.shape: (100,)
ic| neutrino_momenta.shape: (1000,)


END
